In [1]:
#import the library we will be using
import pandas as pd

#find files
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

#read files
school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

#merge the student data with the school data
merged_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
#calculate number of distinct schools
district_summary=pd.DataFrame([{"Total Schools":len(merged_data["school_name"].unique())}])
#calculate number of students
district_summary["Total Students"]=merged_data.shape[0]
#add up the budgets of the schools
district_summary["Total School Budget"]=school_data["budget"].sum()
#calculate average math score
district_summary["Average Math Score"]=merged_data["math_score"].mean()
#calculate average reading score
district_summary["Average Reading Score"]=merged_data["reading_score"].mean()
#calculate percentage of students passing math
district_summary["Percent passing Math"]=len(merged_data.loc[merged_data["math_score"]>=70,"math_score"])/merged_data.shape[0]*100
#calculate percentage of students passing reading
district_summary["Percent Passing Reading"]=len(merged_data.loc[merged_data["reading_score"]>=70,"reading_score"])/merged_data.shape[0]*100
#calculate percentage of students passing math AND reading
district_summary["Percent Passing Overall"]=len(merged_data.loc[((merged_data["reading_score"]>=70) & (merged_data["math_score"]>=70)),"reading_score"])/merged_data.shape[0]*100

#print district summary
district_summary



,Total Schools,Total Students,Total School Budget,Average Math Score,Average Reading Score,Percent passing Math,Percent Passing Reading,Percent Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [3]:
#school_names=school_data["school_name"]
#school_types=school_data["type"]
#copy over several columns from the school data dataframe
school_summary=school_data[["school_name","type","size","budget"]]
#divide budget of each school by the size of the school to get the per student budget
school_summary["Per Student Budget"]=school_summary["budget"]/school_summary["size"]
#calculate average math and reading scores by school
school_means=merged_data.groupby("school_name",sort=False).mean()
#add the average math score to school_summary dataframe.  
#I use .values.tolist() to turn a pandas series (which is indexed by school names) to a list (which is not)
#I did this to fix an error I was having where the dataframe would populate with NaNs instead of numbers.
#I believe the error was due to the indicies not matching.
school_summary["Average Math Score"]=school_means["math_score"].values.tolist()
#add the average reading score to school_summary dataframe
school_summary["Average Reading Score"]=school_means["reading_score"].values.tolist()

#calculate number of students in each school who passed math, divide by number of students per school and multiply by 100
school_summary["Percent Passing Math"]=(merged_data.loc[merged_data["math_score"]>=70,["school_name","math_score"]].groupby("school_name",sort=False).count()["math_score"]/merged_data.groupby("school_name",sort=False).count()["math_score"]*100).values.tolist()
#calculate number of students in each school who passed reading, divide by number of students per school and multiply by 100
school_summary["Percent Passing Reading"]=(merged_data.loc[merged_data["reading_score"]>=70,["school_name","reading_score"]].groupby("school_name",sort=False).count()["reading_score"]/merged_data.groupby("school_name",sort=False).count()["reading_score"]*100).values.tolist()

#calculate number of students in each school who passed both math AND reading, divide by number of students per school and multiply by 100
school_summary["Percent Passing Overall"]=(merged_data.loc[(merged_data["math_score"]>=70)&(merged_data["reading_score"]>=70),["school_name","math_score"]].groupby("school_name",sort=False).count()["math_score"]/merged_data.groupby("school_name",sort=False).count()["math_score"]*100).values.tolist()

#rename columns copied from merged_data to be prettier
school_summary.rename(columns={"school_name": "School Name", "type":"School Type","size":"Total Students","budget":"School Budget"},inplace=True)
#sort the schools so they are in alphabetical order (like the provided answer) instead of the order they appear in the original data
school_summary.sort_values("School Name", inplace=True)
#use School Name to index the list (to match the provided answer)
school_summary.set_index(school_summary["School Name"],inplace=True,drop=True)
#you don't need school name twice
school_summary.drop(columns="School Name",inplace=True)

#format the money amounts in dollars and cents
school_summary["School Budget"]=school_summary["School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"]=school_summary["Per Student Budget"].map("${:,.2f}".format)

#print the summary of schools
school_summary

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [4]:
#print the summary of highest achieving schools
school_summary.sort_values("Percent Passing Overall",ascending=False).head()

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [5]:
#print summary of lowest performing schools
school_summary.sort_values("Percent Passing Overall").head()

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [6]:
math_averages=pd.DataFrame()
#calculate average math scores for each grade at each school
math_averages["9th"]=merged_data.loc[merged_data["grade"]=="9th",:].groupby("school_name").mean()["math_score"]
math_averages["10th"]=merged_data.loc[merged_data["grade"]=="10th",:].groupby("school_name").mean()["math_score"]
math_averages["11th"]=merged_data.loc[merged_data["grade"]=="11th",:].groupby("school_name").mean()["math_score"]
math_averages["12th"]=merged_data.loc[merged_data["grade"]=="12th",:].groupby("school_name").mean()["math_score"]
#print results
math_averages

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [7]:
reading_averages=pd.DataFrame()
#calculate average reading scores for each grade at each school
reading_averages["9th"]=merged_data.loc[merged_data["grade"]=="9th",:].groupby("school_name").mean()["reading_score"]
reading_averages["10th"]=merged_data.loc[merged_data["grade"]=="10th",:].groupby("school_name").mean()["reading_score"]
reading_averages["11th"]=merged_data.loc[merged_data["grade"]=="11th",:].groupby("school_name").mean()["reading_score"]
reading_averages["12th"]=merged_data.loc[merged_data["grade"]=="12th",:].groupby("school_name").mean()["reading_score"]
reading_averages["12th"]=merged_data.loc[merged_data["grade"]=="12th",:].groupby("school_name").mean()["reading_score"]
#print results
reading_averages

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [8]:
#set up the bins and labels
bins=[0,585,630,645,680]
labels=["<$585","$585-630","$630-645","$645-680"]
#get the data
budgetdf=merged_data
#add a column that labels the students by budget category of their school
budgetdf["Spending Range (Per Student)"]=pd.cut(budgetdf["budget"]/budgetdf["size"],bins,labels=labels)

#calculate average math and reading scores by taking all the students in each budget category and averaging the scores of the students
#the provided answers instead average the scores of the schools, which is different methodology
#averaging across schools ignores the fact that each school have a different number of students (one school has more than ten times the students of another!)
budget_summary=pd.DataFrame()
budget_summary["Average Math Score"]=budgetdf.groupby("Spending Range (Per Student)").mean()["math_score"]
budget_summary["Average Reading Score"]=budgetdf.groupby("Spending Range (Per Student)").mean()["reading_score"]

#count how many students in each budget category passed math, reading or both, then divide by the number of students in the category and multiply by 100
#this is also different methodology from what is used in the sample answers
budget_summary["Percent Passing Math"]=(budgetdf.loc[budgetdf["math_score"]>=70,:].groupby("Spending Range (Per Student)").count()["math_score"]/(budgetdf.groupby("Spending Range (Per Student)").count()["math_score"]))*100
budget_summary["Percent Passing Reading"]=budgetdf.loc[budgetdf["reading_score"]>=70,:].groupby("Spending Range (Per Student)").count()["reading_score"]/(budgetdf.groupby("Spending Range (Per Student)").count()["reading_score"])*100
budget_summary["Percent Passing Overall"]=(budgetdf.loc[((budgetdf["math_score"]>=70)&(budgetdf["reading_score"]>=70)),:].groupby("Spending Range (Per Student)").count()["math_score"])/(budgetdf.groupby("Spending Range (Per Student)").count()["math_score"])*100

#print results
budget_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Spending Range (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,90.640704
$585-630,79.982873,82.312643,79.109851,88.513145,70.939239
$630-645,77.821056,81.301007,70.623565,82.600247,58.841194
$645-680,77.049297,81.005604,66.230813,81.109397,53.528791


In [9]:
#set up the bins and labels
bins=[0,1000,2000,5000]
labels=["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
sizedf=merged_data
#add a column that labels the students by size category of their school
sizedf["School Size"]=pd.cut(sizedf["size"],bins,labels=labels)
size_summary=pd.DataFrame()
#calculate average math and reading scores by taking all the students in each size category and averaging the scores of the students
#this yields slightly different results from averaging across schools
size_summary["Average Math Score"]=sizedf.groupby("School Size").mean()["math_score"]
size_summary["Average Reading Score"]=sizedf.groupby("School Size").mean()["reading_score"]

#count how many students in each size category passed math, reading or both, then divide by the number of students in the category and multiply by 100
#still slightly different from averaging across schools
size_summary["Percent Passing Math"]=(sizedf.loc[sizedf["math_score"]>=70,:].groupby("School Size").count()["math_score"]/(sizedf.groupby("School Size").count()["math_score"]))*100
size_summary["Percent Passing Reading"]=sizedf.loc[sizedf["reading_score"]>=70,:].groupby("School Size").count()["reading_score"]/(sizedf.groupby("School Size").count()["reading_score"])*100
size_summary["Percent Passing Overall"]=(sizedf.loc[((sizedf["math_score"]>=70)&(sizedf["reading_score"]>=70)),:].groupby("School Size").count()["math_score"])/(sizedf.groupby("School Size").count()["math_score"])*100

#print results
size_summary


,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


In [10]:
type_summary=pd.DataFrame()

#average the scores of all charter or all district students (not averaging across schools)
type_summary["Average Math Score"]=merged_data.groupby("type").mean()["math_score"]
type_summary["Average Reading Score"]=merged_data.groupby("type").mean()["reading_score"]

#count how many students in each type passed math, reading or both, then divide by the number of students in the type and multiply by 100
#still slightly different from averaging across schools
type_summary["Percent Passing Math"]=(merged_data.loc[merged_data["math_score"]>=70,:].groupby("type").count()["math_score"]/(merged_data.groupby("type").count()["math_score"]))*100
type_summary["Percent Passing Reading"]=merged_data.loc[merged_data["reading_score"]>=70,:].groupby("type").count()["reading_score"]/(merged_data.groupby("type").count()["reading_score"])*100
type_summary["Percent Passing Overall"]=(merged_data.loc[((merged_data["math_score"]>=70)&(merged_data["reading_score"]>=70)),:].groupby("type").count()["math_score"])/(merged_data.groupby("type").count()["math_score"])*100

#print results
type_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878


# Conclusions

In [11]:
#    Two major trends stick out from this data.  First is that school funding 
#seems to be going most to the lower performing schools.  
#This makes sense if you consider that the most at-risk students are 
#the ones most likely to score lower on standardized tests and also need the most support programs.

#    The second trend is that charter schools have higher scores than district schools.  
#This is likely due to a bias in which students attend the charter schools compared to the district schools.  
#Charter schools are better able to select for students who are already going to get higher test scores, 
#leaving district schools with the students likely to get lower scores.